In [1]:
import numpy as np
import  pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import torch 
import sys
sys.path.append('../')
from training.networks import  define_G

import sys
sys.path.append('../')

from training.model import * #Generator,Discriminator
from training.model_unet import * #Generator,Discriminator
from training.database import *
from training.utils import *
# from code.database import ImageDataset

import numpy as np
import gc
import os
import sys
import cv2

import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch
from torchvision.transforms.functional import to_pil_image

import itertools
from PIL import Image

import random

from tqdm import tqdm

In [2]:
class TestOpts():
    def __init__(self,seed=1234):
        self.gan_name="cycle_paired"

        self.fix_seed(seed)
        self.experience_ver="cyclegan_unpaired_ver5.1.0"
        # self.experience_ver="cyclegan_unpaired_ver1.0.0"
        # self.experience_ver="cyclegan_paired_ver1.0.0"
        self.version="0.0.0"
        self.start_epoch = 0
        self.n_epochs = 1
        self.batch_size = 1
        self.dataroot = "../dataset/SUNRGBD/SUNRGBD/kv1/b3dodata/"
        self.lr = 0.0002
        self.decay_epoch = 200
        self.size = (256,256)
        self.domainA_nc = 3
        self.domainB_nc = 3
        self.cpu = False
        self.n_cpu = 0
        self.device_name = "cuda:0" #if torch.cuda.is_available() else "cpu"
        self.device =  torch.device(self.device_name) 
        self.load_weight = True
        self.depth_name="depth_bfx"

    def fix_seed(self,seed):
        # Numpy
        np.random.seed(seed)
        random.seed(seed)
        # Pytorch
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
opt=TestOpts()

In [3]:
# import mlflow

""" --- Initial Setting  ---"""

root_path="../output/"
model_path=root_path+f"model/{opt.experience_ver}/netG_A2B.pth"
model_path_=root_path+f"model/{opt.experience_ver}/netG_B2A.pth"
# model_path=r"latest_net_G_A.pth"
# model_path_=r"latest_net_G_B.pth"
record_path=root_path+f"generate/{opt.experience_ver}/"
analysis_path=root_path+f"analysis/{opt.experience_ver}/"

In [4]:
def get_transform( params=None, grayscale=None, method=Image.BICUBIC, convert=True):
    transform_list = []
    if grayscale:
        transform_list.append(transforms.Grayscale(1))
    
    osize = [opt.load_size, opt.load_size]
    transform_list.append(transforms.Resize(osize, method))
    
    transform_list.append(transforms.RandomCrop(256))
    

    # if not opt.no_flip:
    #     if params is None:
    #         transform_list.append(transforms.RandomHorizontalFlip())
    #     elif params['flip']:
    #         transform_list.append(transforms.Lambda(lambda img: __flip(img, params['flip'])))

    if convert:
        transform_list += [transforms.ToTensor()]
        if grayscale:
            transform_list += [transforms.Normalize((0.5,), (0.5,))]
        else:
            transform_list += [transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    return transforms.Compose(transform_list)

In [16]:
def disnorm(img):
    IMAGENET_MEAN = [0.5, 0.5, 0.5]
    IMAGENET_STD = [0.5, 0.5, 0.5]

    return 0.5*(img + 1.0)
def norm_im(im,cof=1):
    return (im-np.min(im))/(np.max(im)-np.min(im))*cof

def create_image(netG_A2B,net_B2A,input_A,input_B,batch,is_norm=False):
        # Set model input
        real_A = Variable(input_A.copy_(batch['A']))
        # real_B = Variable(input_B.copy_(batch['B']))

        # Generate output
        # fake_B = 0.5*(netG_A2B(real_A).data + 1.0)
        fake_B=tensor2im(netG_A2B(real_A))
        cycle_A=tensor2im(netG_B2A(netG_A2B(real_A)))

        # out_img1 = torch.cat([real_A, fake_B,real_B], dim=2)

        A=tensor2im(real_A)
        B=None#tensor2im(real_B)

        if is_norm:
            return A,B,norm_im(fake_B)
        

    
        return A,B,fake_B,cycle_A

In [6]:
""" --- Call Models ---"""


# 生成器
# netG_A2B = Generator(opt.domainA_nc, opt.domainB_nc)
netG_A2B=define_G(input_nc=3,output_nc=1,ngf=64,netG='resnet_9blocks')
netG_B2A=define_G(input_nc=1,output_nc=3,ngf=64,netG='resnet_9blocks')
# GPU
if not opt.cpu:
    netG_A2B.cuda()
    netG_B2A.cuda()

# 重みパラメータ初期化
# netG_A2B.apply(weights_init_normal)

# 保存したモデルのロード
# state_dict = torch.load(model_path, map_location="cuda:0")
# if hasattr(state_dict, '_metadata'):
#     del state_dict._metadata
# for key in list(state_dict.keys()):  # need to copy keys here because we mutate in loop
#     __patch_instance_norm_state_dict(state_dict, netG_A2B, key.split('.'))
# netG_A2B.load_state_dict(state_dict)

netG_A2B.load_state_dict(torch.load(model_path, map_location="cuda:0"), strict=False)
netG_B2A.load_state_dict(torch.load(model_path_, map_location="cuda:0"), strict=False)

   


# 入出力メモリ確保
Tensor = torch.cuda.FloatTensor if not opt.cpu else torch.Tensor
input_A = Tensor(opt.batch_size, opt.domainA_nc, opt.size[1], opt.size[0])
input_B = Tensor(opt.batch_size, opt.domainB_nc, opt.size[1], opt.size[0])
target_real = Variable(Tensor(opt.batch_size).fill_(1.0), requires_grad=False)
target_fake = Variable(Tensor(opt.batch_size).fill_(0.0), requires_grad=False)

# 過去データ分のメモリ確保
# データローダー
transforms_ = [ 
                transforms.Resize((int(opt.size[0]),int(opt.size[1])), Image.BICUBIC), 
                # transforms.RandomCrop(opt.size), 
                # transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize((0.5,), (0.5,)) 
                ]

dataset=ImageDataset(opt,root=opt.dataroot, unaligned=False,limit=None)
dataloader = DataLoader(dataset, 
                        batch_size=opt.batch_size, shuffle=True, num_workers=opt.n_cpu)

#Dataset for sampling 
sample_images=[dataset[i] for i in range(100) ]

print("num dataloader= {}".format(len(dataloader)))

#Release Memory
# del netD_A,netD_B,fake_A_buffer,fake_B_buffer
# gc.collect()


#Save generate image from netG
images=create_image(netG_A2B,netG_B2A,input_A,input_B,sample_images[0])

# mlflow.end_run()


initialize network with normal
initialize network with normal
num dataloader= 554


In [15]:
import shutil
results={"color":[],"depth":[],"fake":[]}
folders=os.listdir(opt.dataroot)
for i in tqdm(range(len(dataset))):
    batch=dataset[i]
    _save_path=f"{opt.dataroot}{folders[i]}/{opt.gan_name}/"
    if not os.path.exists(_save_path):
        os.mkdir(_save_path)
    else:
        shutil.rmtree(_save_path)
        os.mkdir(_save_path)
    imgs=create_image(netG_A2B,netG_B2A,input_A,input_B,batch)
    cv2.imwrite(f"{_save_path}fake_{i}.png",imgs[2])

    

100%|██████████| 554/554 [00:53<00:00, 10.27it/s]


In [8]:
pathes=np.loadtxt(r"..\dataset\SUNRGBD\SUNRGBD\kv2\kinect2data_segmentation\test_images_path.txt",dtype=str)
pathes[0][:pathes[0].rfind("/image")]

'SUNRGBD/kv2/kinect2data/000002_2014-05-26_14-23-37_260595134347_rgbf000103-resize'

In [18]:
results={"color":[],"depth":[],"fake":[]}
trans= transforms.Compose(transforms_)
for i in tqdm(range(len(pathes))):
    _p="../dataset/SUNRGBD/"+pathes[i]
    # batch=dataset[i]
    color=trans(Image.open(_p).convert('RGB'))
    _r=pathes[i][:pathes[i].rfind("/image")]
    _save_path=f"../dataset/SUNRGBD/{_r}/{opt.gan_name}/"
    if not os.path.exists(_save_path):
        os.mkdir(_save_path)
    imgs=create_image(netG_A2B,netG_B2A,input_A,input_B,{"A":color,"B":None})
    cv2.imwrite(f"{_save_path}fake_{i}.png",imgs[2])

100%|██████████| 5050/5050 [07:33<00:00, 11.14it/s]
